<i>Written by: Laura Brin, Sandra Alex & Annabell Rodriguez, On behalf of Norquest College Institute for the CMPT-3510 Machine Learning I Fall course</i>

# House Price Regression

We'll be using a labeled dataset that contain features representing house characteristics like measurements, number of rooms, location and price. The last one (price) is the class that we are looking to predict based on the rest of the features. The data was collected from King County, USA. This dataset is from Kaggle. More information about the dataset can be found here: https://www.kaggle.com/datasets/harlfoxem/housesalesprediction. 

In [ ]:
#Crucial data processing and analysis libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import datetime as dt


# Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# loading the house sales csv data
df = pd.read_csv("kc_house_data.csv")

#Others
from IPython.display import Image 

# This makes it so we are able to see 100 rows when displaying the data
pd.set_option("display.max_rows", 100)

### Problem definition

The tendency of the house prices in United States is to grothw. That have been the trend over the last years. In our dataset is difficult to visualize this pattern because it only covers 2 years, from May 2014 to May 2015.

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df['month'] = df['date'].to_numpy().astype('datetime64[M]')
plt.figure(figsize=(15,6))
out_price = df.groupby('month')['price'].median().reset_index(name ='Price')
sns.lineplot(x='month',y='Price',data=out_price)
plt.title("Price vs Date")
plt.xlabel("Date")
plt.ylabel("Price")
plt.show()

But, we can see this trend analyzing the data provided by Fred (https://fred.stlouisfed.org/series/MSPUS)

In [ ]:
df_fred = pd.read_csv("MSPUS.csv")

plt.figure(figsize=(15,6))
sns.lineplot(x='observation_date',y='MSPUS',data=df_fred)
plt.title("Price vs Date")
plt.xlabel("Date")
plt.ylabel("Price")
every_nth_xtick = 25
plt.xticks(np.arange(0, len(df_fred)+10, every_nth_xtick))
plt.show()

    For people trying to buy or sell a house, this can be problematic. The first ones need to know if the price of the house is fair according to the changes in the market and some other factors that influence its price like physical characteristics (number of rooms, size, location, etc) or the market fluctuation due to inflation, economical crisis, among others. For the second case, when you are trying to sell, you want to get the most from your house, then it is important to take all the factors mentioned before into account. Then having a model that helps people to predict the price of a particular house can be a solution for many. 
    
    This dataset contains 19 features. These are:

date - Date of the home sale<br>
bedrooms - Number of bedrooms<br>
bathrooms - Number of bathrooms, where .5 accounts for a room with a toilet but no shower<br>
sqft_living - Square footage of the apartments interior living space<br>
sqft_lot - Square footage of the land space<br>
floors - Number of floors<br>
waterfront - A dummy variable for whether the apartment was overlooking the waterfront or not<br>
view - An index from 0 to 4 of how good the view of the property was<br>
condition - An index from 1 to 5 on the condition of the apartment<br>
grade - An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design<br>
sqft_above - The square footage of the interior housing space that is above ground level<br>
sqft_basement - The square footage of the interior housing space that is below ground level<br>
yr_built - The year the house was initially built<br>
yr_renovated - The year of the house’s last renovation<br>
zipcode - What zipcode area the house is in<br>
lat - Lattitude<br>
long - Longitude<br>
sqft_living15 - The square footage of interior housing living space for the nearest 15 neighbors<br>
sqft_lot15 - The square footage of the land lots of the nearest 15 neighbors<br>

Every observation in the dataset represents a house sold in that specific date. The date granularity is daily, it contains the day, month and year in which the house was sold, which means that for a single day we can have multiple records. As we mentioned before, the years represented are 2014 and 2015.

### Data Cleaning-Laura

sapce saver

In [ ]:
#code saver

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

**Cleaning Notes**: <br>
need to change date- strip T00 and convert to_datetime <br>
yr_renovated change 0 to NaN <br>
waterfront to binary <br>
grade to categories then encode <br>
sqft_living, sqft_lot, sqft_above, sqft_below, sqft_living15, sqft_lot15 could normalize <br>
condition and view are both categorical, but as they are currently numerical and ordinal there is no needed cleaning



date:

In [ ]:
df_clean=df.copy()
df_clean["date"].astype("string").str.rstrip("T000000") 
df_clean["date"]=df_clean["date"].values.astype("datetime64[D]")
df_clean.dtypes

yr_renovated: NaN for missing years

In [ ]:
df_clean["yr_renovated"].replace(0,np.NaN, inplace=True)

waterfront:

In [ ]:
df_clean["waterfront"].unique()

In [ ]:
df_clean["waterfront"]=df_clean["waterfront"].astype(bool)

grade: categorize and encode

In [ ]:
df_clean["grade"].unique()

In [ ]:
df_clean["grade"]=pd.cut(df_clean["grade"],bins=[1,4,11,13],labels=["low","average","high"])

In [ ]:
df_clean=pd.get_dummies(df_clean, columns=["grade"],prefix="grade", drop_first=True)

In [ ]:
df_clean.head()

In [ ]:
df_clean.dtypes

**remove after features described**
id - Unique ID for each home sold
date - Date of the home sale
price - Price of each home sold
bedrooms - Number of bedrooms
bathrooms - Number of bathrooms, where .5 accounts for a room with a toilet but no shower
sqft_living - Square footage of the apartments interior living space
sqft_lot - Square footage of the land space
floors - Number of floors
waterfront - A dummy variable for whether the apartment was overlooking the waterfront or not
view - An index from 0 to 4 of how good the view of the property was
condition - An index from 1 to 5 on the condition of the apartment,
grade - An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design.
sqft_above - The square footage of the interior housing space that is above ground level
sqft_basement - The square footage of the interior housing space that is below ground level
yr_built - The year the house was initially built
yr_renovated - The year of the house’s last renovation
zipcode - What zipcode area the house is in
lat - Lattitude
long - Longitude
sqft_living15 - The square footage of interior housing living space for the nearest 15 neighbors
sqft_lot15 - The square footage of the land lots of the nearest 15 neighbors

### Data Visualization

In [ ]:
#df_clean.boxplot(column="price")
#df_clean.boxplot(column="bedrooms")
#df_clean.boxplot(column="bathrooms")
#df_clean.boxplot(column="sqft_living")
#df_clean.boxplot(column="sqft_lot")
#df_clean.boxplot(column="floors")
#df_clean.boxplot(column="sqft_above")
#df_clean.boxplot(column="sqft_basement")
#df_clean.boxplot(column="yr_built")
#df_clean.boxplot(column="sqft_living15")
#df_clean.boxplot(column="sqft_lot15")

**insights from boxplots** <br>
All features except yr_built and floors showed significant collective outliers. <br> 
Bedrooms had a single global outlier >30. sqft_living has a potential global outlier at 12000 sqft.  <br>
Not checked: lat, long and zipcode as they are geographic location features. <br>
Not checked: waterfront as it is binary; condition and grade are catagorical; view is a 1-5 scale; yr_renovated NaN values <br>


In [ ]:
features=df_clean.columns
features=features.drop(["id","date","price","grade_average","grade_high"])
graph=1
for col in features:
    fig=plt.scatter(df_clean["price"],df_clean[col])
    plt.xlabel("House Price")
    plt.ylabel(f"{col}")
    plt.title(f"Figure {graph}:House Price vs {col}")
    plt.show()
    graph+=1

**Insights from scatterplots**: <br>
living space square footage appears to have a lower bound where house price will not exceed $200/sqft <br>
upper floor sqft has obvious correlation with total living space sqft <br>
appears to be a positive correlation with condition of house and price (at least at higher prices) <br>
view rating and waterfront did not have obvious strong coorelation with house price. <br>
There appears to be a specific lat-long that correlated to the most expensive properties (Bellevue neighbourbhood in Seattle).

sqft living and sqft lot each have 1 global outlier to remove. sqft_lot15 has 2 global outliers to remove


In [ ]:
corr=df_clean.corr()

correlation_heatmap=plt.figure(num=None, figsize=(20,20))
correlation_heatmap=sns.heatmap(data=corr,annot=True, fmt='.2f').set(title="Figure 18:Heatmap of Correlation for Housing features")
sns.set(font_scale=1.4)

correlation_heatmap

**Insights from correlation plot** <br>
Price is moderately correlated with sqft_living, sqft_above and sqft_living15. Bedrooms and bathrooms have low correlation with price but moderate correlation with sqft_living. View has a higher correlation than waterfront. While the latitude and longitude had clear patterns in the scatterplots, only the latitue shows a minor correlation. 

From data visualizations the following cleaning is required: <br>
* remove bedroom, sqft_living, sqft_lot, and sqft_lot15 outliers
* normalize lat and long values

In [ ]:
df_clean.drop(df_clean[df_clean["bedrooms"]>30].index, inplace=True)
df_clean.drop(df_clean[df_clean["sqft_living"]>13000].index, inplace=True)
df_clean.drop(df_clean[df_clean["sqft_lot"]>1500000].index, inplace=True)
df_clean.drop(df_clean[df_clean["sqft_lot15"]>800000].index, inplace=True)

In [ ]:
scaler=MinMaxScaler()
lat_array=df_clean["lat"].values.reshape(-1,1)
long_array=df_clean["long"].values.reshape(-1,1)

df_clean["lat_norm"]=normalize(lat_array, axis=0)
df_clean["long_norm"]=normalize(long_array, axis=0)

In [ ]:
corr2=df_clean.corr()

correlation_heatmap2=plt.figure(num=None, figsize=(20,20))
correlation_heatmap2=sns.heatmap(data=corr2,annot=True, fmt='.2f').set(title="Figure 19: Heatmap of Correlation for Housing features")
sns.set(font_scale=1.4)

correlation_heatmap2

In [ ]:
df_clean.head()

normalizing latitude did not have any markable result, normalizing longitude did have an effect of increasing correlation with price but only to low levels <b>


**Reminder**
column grade needs to be removed from dataset used for regression model 

could also seperate data into 2 different models- high price and low price, so that the high outliers do not skew the data (<4M, >4M)?

### Regression

KNN Regression: target price will be predicted by using interpolation of the price of the nearest neighbours
Decision Tree Regression: target price will be predicted based on a tree 

Initializing Classification Models

In [ ]:
tree=DecisionTreeRegressor()
knn=KNeighborsRegressor()
bayes=BayesianRidge()

Spliting Data

In [ ]:
df_regress=df_clean.copy()
df_regress.drop(["id","lat_norm","long_norm","month"], inplace=True, axis=1)
df_regress["yr_renovated"].replace(np.NaN, pd.NaT, inplace=True)
df_regress["date"]=df_regress["date"].map(dt.datetime.toordinal)

X_clean = df_regress.loc[:, df_regress.columns != 'price']
y_clean = df_regress.loc[:, df_regress.columns == 'price']

In [ ]:
X_split, X_valid, y_split, y_valid = train_test_split(X_clean,y_clean, train_size=0.8, random_state=42)
X_train, X_test, y_train, y_test = train_test_split( X_split, y_split, train_size=0.75, random_state=42)

KNN Regression

In [ ]:
knn.get_params()

In [ ]:
#parameters to test
n_neighbors= [3,5,9,33,66,99]
weights=["uniform","distance"]

k_random_grid={"n_neighbors":n_neighbors, "weights":weights}

In [ ]:
#knn.fit(X_train,y_train)

In [ ]:
knn=RandomizedSearchCV(estimator = knn, param_distributions = k_random_grid, n_jobs = -1)

Decision Tree Regression

In [ ]:
tree.get_params()

In [ ]:
#parameters
min_samples_split = [2, 5, 10]
max_features= ["sqrt","log2",None]
ccp_alpha=[0.01,0.02,0.03]
random_grid = {'min_samples_split': min_samples_split,
                    "max_features": max_features, "ccp_alpha":ccp_alpha}

In [ ]:
tree=GridSearchCV(estimator = tree, param_grid = random_grid, n_jobs = -1)

In [ ]:
#tree.fit(X_train,y_train)

In [ ]:
tree.best_params_